In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|default_exp agents.dqn.target

In [ ]:
#|export
# Python native modules
from copy import deepcopy
from typing import Optional,Callable,Tuple
# Third party libs
import torchdata.datapipes as dp
from torchdata.dataloader2.graph import traverse_dps,DataPipe
import torch
from torch import nn,optim
# Local modules
from fastrl.pipes.core import find_dp
from fastrl.memory.experience_replay import ExperienceReplay
from fastrl.loggers.core import BatchCollector,EpochCollector
from fastrl.learner.core import LearnerBase,LearnerHead
from fastrl.agents.dqn.basic import (
    LossCollector,
    RollingTerminatedRewardCollector,
    EpisodeCollector,
    StepBatcher,
    TargetCalc,
    LossCalc,
    ModelLearnCalc,
    DQN,
    DQNAgent
)

In [ ]:
#|hide
import logging
from fastrl.core import default_logging

In [ ]:
#|hide
logging.basicConfig(**default_logging())

# DQN Target
> DQN that uses snapshots from the NN module to stabilize training



## Training DataPipes

In [ ]:
#|export
class TargetModelUpdater(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe=None,target_sync=300):
        self.source_datapipe = source_datapipe
        if source_datapipe is not None:
            self.learner = find_dp(traverse_dps(self),LearnerBase)
            self.learner.target_model = deepcopy(self.learner.model)
        self.target_sync = target_sync
        self.n_batch = 0
        
    def reset(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        self.learner.target_model = deepcopy(self.learner.model)
        
    def __iter__(self):
        if self._snapshot_state.NotStarted: 
            self.reset()
        for batch in self.source_datapipe:
            if self.n_batch%self.target_sync==0:
                self.learner.target_model.load_state_dict(self.learner.model.state_dict())
            self.n_batch+=1
            yield batch

In [ ]:
#|export
class TargetModelQCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe=None):
        self.source_datapipe = source_datapipe
        
    def __iter__(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        for batch in self.source_datapipe:
            self.learner.done_mask = batch.terminated.reshape(-1,)
            with torch.no_grad():
                self.learner.next_q = self.learner.target_model(batch.next_state)
            self.learner.next_q = self.learner.next_q.max(dim=1).values.reshape(-1,1)
            self.learner.next_q[self.learner.done_mask] = 0 
            yield batch

In [ ]:
#|export
def DQNTargetLearner(
    model,
    dls,
    logger_bases:Optional[Callable]=None,
    loss_func=nn.MSELoss(),
    opt=optim.AdamW,
    lr=0.005,
    bs=128,
    max_sz=10000,
    nsteps=1,
    device=None,
    batches=None
) -> LearnerHead:
    learner = LearnerBase(model,dls=dls[0])
    learner = BatchCollector(learner,batches=batches)
    learner = EpochCollector(learner)
    if logger_bases: 
        learner = logger_bases(learner)
        learner = RollingTerminatedRewardCollector(learner)
        learner = EpisodeCollector(learner)
    learner = learner.catch_records()
    exp_replay = ExperienceReplay(learner,bs=bs,max_sz=max_sz)
    learner = StepBatcher(exp_replay,device=device)
    learner = TargetModelQCalc(learner)
    learner = TargetCalc(learner,nsteps=nsteps)
    learner = LossCalc(learner,loss_func=loss_func)
    learner = ModelLearnCalc(learner,opt=opt(model.parameters(),lr=lr))
    learner = TargetModelUpdater(learner,target_sync=300)
    if logger_bases: 
        learner = LossCollector(learner).catch_records()
    learner = LearnerHead(learner,model)
    return learner,exp_replay

Try training with basic defaults...

In [ ]:
from fastrl.loggers.vscode_visualizers import VSCodeDataPipe
from fastrl.envs.gym import GymDataPipe
from fastrl.loggers.core import ProgressBarLogger
from fastrl.dataloading.core import dataloaders

In [ ]:
# #|eval:false
# # Setup Loggers
# def logger_bases(pipe):
#     pipe = pipe.dump_records()
#     pipe = ProgressBarLogger(pipe)
#     return pipe
# # Setup up the core NN
# torch.manual_seed(0)
# model = DQN(4,2)
# # Setup the Agent
# agent = DQNAgent(model,do_logging=True,max_steps=4000)
# # Setup the DataBlock
# dls = dataloaders(
#     GymDataPipe(
#         ['CartPole-v1']*1,
#         agent=agent,
#         nsteps=1,
#         nskips=1,
#         firstlast=False,
#         bs=1
#     )
# )
# # Setup the Learner
# learner,exp_replay = DQNTargetLearner(
#     model,
#     dls,
#     logger_bases=logger_bases,
#     bs=128,
#     max_sz=100_000,
#     batches=1000
# )
# learner.fit(3)
# # learner.fit(25)

The DQN learns, but I wonder if we can get it to learn faster...

In [ ]:
# logging.getLogger('fastrl.loggers.core').setLevel('DEBUG')

In [ ]:
#|eval:false
# Setup Loggers
def logger_bases(pipe):
    pipe = pipe.dump_records()
    pipe = ProgressBarLogger(pipe)
    return pipe
# Setup up the core NNa
torch.manual_seed(0)
model = DQN(4,2)
# Setup the Agent
agent = DQNAgent(model,do_logging=True,max_steps=5000)
# Setup the DataBlock
train_pipe = GymDataPipe(
    ['CartPole-v1']*1,
    agent=agent,
    nsteps=2,
    nskips=2,
    firstlast=True,
    bs=1
)
valid_pipe = GymDataPipe(
    ['CartPole-v1']*1,
    agent=agent,
    nsteps=2,
    nskips=2,
    firstlast=True,
    bs=1,
    n=100,
    include_images=True
)
valid_pipe = VSCodeDataPipe(valid_pipe)
dls = dataloaders((train_pipe,valid_pipe))
# Setup the Learner
learner,exp_replay = DQNTargetLearner(
    model,
    dls,
    logger_bases=logger_bases,
    bs=128,
    max_sz=100_000,
    nsteps=2,
    lr=0.001,
    batches=1000
)
# learner.fit(3)
learner.fit(5)

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1000 [00:00<?, ?it/s]

loss,episode,rolling_reward,epoch,batch,epsilon
0.08227259,102,16.84,0,999,0.6308
0.66846615,240,10.52,1,999,0.2714
1.1289164,415,9.20,2,999,0.0200
1.6646713,596,9.30,3,999,0.0200
1.5626208,781,9.30,4,999,0.0200


In [ ]:
exp_replay.show()

<fastrl.memory.memory_visualizer.MemoryBufferViewer>

In [ ]:
#|hide
#|eval:false
learner.validate(1)

IndexError: tuple index out of range

In [ ]:
#|hide
#|eval: false
!nbdev_export